<a href="https://colab.research.google.com/github/patricksabry/UTS_ML2019_ID98106545/blob/master/A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plain text link to the Notebook on GitHub: 

https://github.com/patricksabry/UTS_ML2019_ID98106545/blob/master/A3.ipynb

# Assignment 3 Take Home Exam

##Question 3:

**Marketing or advertising companies would be very interested in being able to
predict whether a Twitter message will spread as a meme or not, and even better,
construct it so that it will spread. Why is this a hard problem to solve? Describe two
approaches using data analytics to predict whether a tweet will go viral or not. How
would you validate these approaches? Discuss the ethical and social consequence
of this study.**

# Introduction

What truly constitutes a meme? 
The enormous popularity and pervasive use of social networks has given rise to a new age of online media hyperconsumption in the form of text, images and videos. Large scale social media platforms such as twitter have become formidable mediums for wide scale information propagation. Any user can choose to share their opinion on a product or post a humerous picture with the chance of it becoming a viral sensation on twitter. It is however that very definition of virality that is so difficult to qualify and break down into concrete quantitative attributes. A meme was first defined as a unit of cultural transmission (Dawkins, 1976). Accordingly, the success of a meme in respect to virality is arguably contingent on a plethora of sociocultural and geo-political factors at a given time. Memes tend to provide an opinionated commentary on the state of current affairs and the flaws of the human condition, typically formed on the tenants of humour and self-deprication. It is thus hard to quantify an object which is inherently so dynamic and subjective when approaching this topic from a data analytics perspective. Social network link analysis is an approach which is used to investigate social structures through the analysis of their structural features alone such as nodes and links. Similarly, recent advancements in data analytics have shifted focus away from the structural features of social networks and towards analysis and application of a combination of structural, content-based and latent (sentiment) features to predict tweet virality. This paper will strive to evaluate these techniques as well as present a case study of a state-of-the-art implementation of a scalable viral tweet prediction system.

# Social Network Analysis vs. Machine Learning

Social network analysis (SNA) is formally known as the process of investigating social structures through the use of networks and graph theory (Otte, et al.,2002). This approach is used as a statistical tool to help understand the dynamics of information propagation in social networks, pertaining to a multitude of applications such as political influence, marketing campaigns and of course memes. This technique is predicated on graph theory, as each twitter user would be represented as a node and the structural (e.g. followers) and sentimental (e.g. favourites) features relating each node can be embodied by the edges connecting the nodes. The pitfall of SNA is the narrow scope of the feature space used in graph based statistical modelling. Relying on solely structural features limits an SNA model's capacity to infer sentiment. "Deciding how many retweets make a tweet 'viral' depends on the underlying social network and on the topic of the tweet" (L Corvacchiola et al., 2017). On the other hand, majority of machine learning approaches allow for the utiliztion of a much more flexible feature space consisting of a composition of structural, user-content and latent features. Using not only structural but also latent features to quantify user sentiments in a social network is paramount to achieving a highly accurate tweet popularity (retweet) prediction (M.Jenders, G.Kasneci, F.Naumann., 2013). A number of different machine learning techniques could be leveraged to learn said feature space, most notably Deep Learning, Generalised Linear Modelling and Gradient Boosting. A state-of-the-art implementation of a scalable Gradient Boosting algorithm which accurately predicts tweet virality is evaluated in the case study below.


# Case Study -  Scalable Tweet Virality Prediction Using Gradient Boosted Tree Algorithm and Spark


Kowalczyk and Larsen of the Microsoft Development Center proposed an interesting solution to privacy-compliant virality prediction using a Gradient Boosted Tree (GBT) based algorithm. This approach leverages a mixture of content, temporal and sentiment based features extracted via the Twitter API to efficiently generate predictions at scale. The solution boasts an intricate distributed data processing framework which pipelines the solution into three distinct steps; data ingestion, feature extraction and finally model fitting.




![Figure 1](https://i.ibb.co/QkP7P65/a3-fig1.png)

**Figure 1** - Solution Overview Diagram of GBT based Twitter virality prediction system. (D. K. Kowalczyk, J. Larsen. 2019)


### Data Ingestion
Data privacy compliance is imperative when dealing with big social data. Stringent regulations such as the General Data Protection Regulation (GDPR, 2018) in Europe dictate how certain features are collected and more importantly how these features are represented in an analytical model. This solution addresses privacy concerns by integrating Twitter's Compliance Firehose API into its data ingestion process. The Firehose API allows for the cleaning and abstraction of large volumes of Twitter stream data based on user requests on the Twitter platform itself. If a user desires to delete a tweet on the Twitter platform, that deletion request is automatically cascaded to this project's datastore and is deleted from the dataset too. Furthermore, the Twitter Engagement Totals API is also called during data collection, which automatically retrieves all of the retweets and favourites ever registered against a particular tweet. This API facilitates the efficient collection of unique content and thus reduces the document volume for the task by half (D.K Kowalczyk, J.Larsen. 2019) in the interest of space and time complexity at scale. Subsequently, this stream data is indexed into partitions in a distributed Azure data store for further pre-processing.

### Sentiment Analysis

Sentiment analysis is arguably the crux of the problem domain when it comes to predicting virality of user content on social media. This particular implementation leveraged the Text Analytics API from Microsoft Cognitive Services. The API houses a pre-trained algorithm which returns a continuous value sentiment score between 0 and 1 for each document in a given corpus, where 1 is most positive and 0 is most negative. This API is a paid off-the-shelf service, and thus raises concerns about reproducibility and flexibility in feature engineering.

### Modeling
Despite Deep neural networks having proven advantages for feature extraction and inference in high dimensionality unstructured data (Samek, W., Wiegand, T., M¨uller, K.R. 2017), such networks inherently have a low explainability of results due to the ambiguities of their internal representations (Hinton et al., 1985). This is why the lightGBM implementation of the GBT algorithm was used in this application, providing explainable, scalable GPU accelerated training of an ensembled regression tree-based model. Using Apache spark, the final feature space is aggregated from distributed slave nodes by the master node for fitting and evaluation on a parallel GPU implementation. A loss function is subsequently minimised in respect to an RMSE error measure in order to penalize large errors more aggresively.

### Evaluation
This system was validated on the MBI, T2015 and T2016 datasets consisting of millions of unique tweets. The main metric used to compare the results of the GBT model to previous state-of-the-art models is the Spearman Rank correlation coefficient. This coefficient assesses the similraity between two rank values as a continuous value between -1 (negative correlation) and 1 (positive correlation). Alongside the SpearmanR measure, $R^{2}$ and RMSE error measures were also used to measure the difference between the expected popularity of a tweet (number of retweets) and the model's prediction. Kowalczyk & Larsen's GBT model outperformed the other state-of-the-art models by more than 37% across all of the aforementioned datasets (D. K. Kowalczyk, J. Larsen. 2019).

# Ethical and Social Implications

The productization of such a predictive system can be stagnated by underlying user data privacy requirements. The system must adhere to stringent local and international privacy regulations such as Europe's GDPR and Australia's federal Privacy Act. The Microsoft case study above ingeniously addresses this ethical concern by discarding unnecessary private user data, storing user data in a central data store and subsequently anonymizing that data and pre-processing it in-memory as opposed to directly in the central data store. Privacy-compliant architectures such as this one should represent the future of big social data endeavours as privacy concerns continue to escalate in the wake of anti-trust probes conducted by the US government against technology giants such as Facebook, Google, Apple and Amazon.

Furthermore, ever since the dawn of the internet and social media, memes have been veiled in a mist of controversy. Internet memes are polarizing because they have the potential to spread positive humor, or conversely spread ethically fraught sentiments. Digital memes travel with high velocity and are prone to be decontextualised quickly (W Phillips, 2016). In the wrong hands they have the capacity to fuel malicious defamation campaigns, circulate political propaganda and ultimately cause civil unrest. These issues are important to consider when studying the constituent attributes of viral memes when attempting to reverse engineer them. What is the purpose of predicting meme virality? will the use case be limited to harmless product advertisement campaigns? these are the pertinent questions which must be asked when building and using such a system. Despite the obvious potential for misuse, I truly believe such predictive systems could assist in engineering memes to spread awareness of injustice, promote free enterprise and above all put smiles on faces. 

# Conclusion

Despite Generalised Linear Models and Gradient Boosting techniques outperforming certain Naive Bayes techniques (M.Jenders, G.Kasneci, F.Naumann., 2013), it is appropriate to conclude that a sophisticated, hybrid feature space is the most important component to solving this predictive problem. Drawing from the results of both (D. K. Kowalczyk, J. Larsen. 2019) and (M.Jenders, G.Kasneci, F.Naumann., 2013), structural or latent features alone will not yield desirable results in this problem domain. Rather, a mixture of structural, content based and sentimental features must be fit on a model that generalises well and avoids simplifying independence assumptions (M.Jenders, G.Kasneci, F.Naumann., 2013). I believe the field of natural language processing is still in its infancy and advancements in NLP will inevitably lead to more human-like inference and interpretation of sentiment in not only text data but also image and video data. Such advancements will allow this field of research to deter away from binary sentiment analysis (positive/negative) and more towards the multi-faceted emotional inference that human cognition affords. Once machine inference achieves such performance levels I am confident actors can reverse engineer 'memes' with the intent of achieving virality. 

# References

* Maximilian Jenders , Gjergji Kasneci , Felix Naumann, Analyzing and predicting viral tweets, Proceedings of the 22nd International Conference on World Wide Web, May 13-17, 2013, Rio de Janeiro, Brazil [Accessed 4 October 2019]
* Castaño Díaz, C., 2013. Defining and characterizing the concept of Internet Meme. CES Psychology Magazine, [Online]. 6, 82-104. Available at: http://www.scielo.org.co/pdf/cesp/v6n2/v6n2a07.pdf [Accessed 3 October 2019].
* Corvacchiola et al., 2017. Estimating the Spreading of Viral Threads on Twitter. In 3rd International Workshop on Knowledge Discovery on the WEB. Cagliari, Italy, Sep 11 to 12, 2017: CEUR-WS.org. 1-7.
* Kowalczyk, D., Larsen, J. (2019). Scalable Privacy-Compliant Virality Prediction on Twitter?. In 2nd Workshop on Affective Content Analysis (AffCon 2019) co-located with Thirty-Third AAAI Conference on Artificial Intelligence (AAAI 2019). Honolulu, USA, Jan 27, 2019: CEUR-WS.org. 1-15.
* Samek, W., Wiegand, T., M¨uller, K.R. Explainable Artificial Intelligence: Understanding, Visualizing and Interpreting Deep Learning Models (aug 2017), [Online]. Available at: http://arxiv.org/abs/1708.08296 [Accessed 5 October 2019].
* Microsoft. 2019. What is the Text Analytics API?. [ONLINE] Available at: https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/overview. [Accessed 7 October 2019].
* healthychildren.org. 2018. Dangerous Internet Challenges – Understanding their Appeal. [ONLINE] Available at: https://www.healthychildren.org/English/family-life/Media/Pages/Dangerous-Internet-Challenges.aspx. [Accessed 7 October 2019].
* ethics.org.au. 2016. The complex ethics of online memes. [ONLINE] Available at: https://ethics.org.au/the-complex-ethics-of-online-memes/. [Accessed 8 October 2019].